# Introduction to Bigquery




Extract the following fields:
name, gender, year and number

For states that share a border with Mexico:
CA, AZ, NM or TX


In [ ]:
SELECT
 name,
 gender,
 year,
 number
FROM `bigquery-public-data.usa_names.usa_1910_current`
WHERE
 state IN ("CA", "AZ", "NM", "TX")


In [ ]:
SELECT
 name,
 gender,
 year,
 number
FROM `bigquery-public-data.usa_names.usa_1910_current`
WHERE
 state = "CA"
 OR state = "AZ"
 OR state = "NM"
 OR state = "TX"


# Practice Aggregations, Grouping and Having

Unique count of names per state.

We need to group by state column ("Group By" fields included in SELECT, but no in the aggregation COUNT,SUM, AVG, etc)

In [ ]:
SELECT
 COUNT(distinct name) as total_of_name,
 state
FROM `bigquery-public-data.usa_names.usa_1910_current`
WHERE
 state = "CA"
 OR state = "AZ"
 OR state = "NM"
 OR state = "TX"
GROUP BY state



Lets get all the states that have more than 20000 unique names

use COUNT to see how many unique names,
change the limit to 10000

In [ ]:
SELECT
 state
 # add this line below to see how many different names there are per state
 #,count(distinct name) as total_name
FROM `bigquery-public-data.usa_names.usa_1910_current`
GROUP BY state
HAVING COUNT(distinct name) > 20000


Using `bigquery-public-data.usa_names.usa_1910_current`, answer the following:

What are the all time most common names?
*The field number gives the occurrences of a given name*

What are the all time most common female names?


In [ ]:
SELECT
 name,
 sum(number) as total_number
FROM `bigquery-public-data.usa_names.usa_1910_current`
GROUP BY name
ORDER BY total_number DESC

In [ ]:
# filtering by female names

SELECT
 name,
 sum(number) as total_number
FROM `bigquery-public-data.usa_names.usa_1910_current`
WHERE gender = "F"
GROUP BY name
ORDER BY total_number DESC


Using `bigquery-public-data.usa_names.usa_1910_current`, answer the following:

Are there more female or male names?

*We only need the field name and gender for this query*



In [ ]:
SELECT
 gender,
 count(distinct name) as total_name
FROM `bigquery-public-data.usa_names.usa_1910_current`
GROUP BY gender


# Practice with JOINS


Which names are both male and female?
Use a WITH statement for this query

My solution would be to make 2 CTE one to get only get male names THEN another to only get females name
Before running a FULL OUTER JOIN
And including a WHERE to ensure that we only get name that share a similar gender


In [ ]:
WITH male_name AS (
   SELECT DISTINCT
       name,
       gender
   FROM `bigquery-public-data.usa_names.usa_1910_current`
   WHERE
       gender = "M"
),
female_name AS (
   SELECT DISTINCT
       name,
       gender
   FROM `bigquery-public-data.usa_names.usa_1910_current`
   WHERE
       gender = "F"
)

SELECT
   -- m.name as m_name,
   -- m.gender as m_gender,
   -- f.name as f_name,
   -- f.gender as f_gender
   COALESCE(m.name, f.name) AS name
FROM male_name AS m
FULL OUTER JOIN female_name AS f
   ON f.name = m.name
WHERE
   m.gender IS NOT NULL
   AND f.gender IS NOT NULL
ORDER BY m.name, f.name


To finish this section, let's JOIN our USA Names tables with another public dataset available in BQ, **bigquery-public-data.samples.natality** contains all United States births registered in the 50 States, the District of Columbia, and New York City from 1969 to 2008. Take a minute to explore this table: schema, preview data.

We work with data for the state of Texas in the year 2000, male names.

Answer this:

If we aggregate the top 10 most common male names in Texas born in 2000, which % this number represents for the total number of births in Texas in 2000?

(The answer is ~ 12%)

In [ ]:
# let's check the top 10 male names in Texas (TX) in 2000:

SELECT
name,
sum(number) as total_by_name
FROM `bigquery-public-data.usa_names.usa_1910_current`
WHERE gender = "M" AND state = "TX" AND year = 2000
GROUP BY name
ORDER BY total_number DESC
LIMIT 10

In [ ]:
# now let's take a look at the number of births for males in TX in 2000

SELECT COUNT(*) as number_births
FROM `bigquery-public-data.samples.natality`
WHERE year = 2000 AND state = 'TX' AND is_male


In [ ]:
WITH male_names AS (
    # top 10 male names in Texas in 2000
    SELECT
    name, state,year,
    sum(number) as total_by_name
    FROM `bigquery-public-data.usa_names.usa_1910_current`
    WHERE gender = "M" AND state = "TX" AND year = 2000
    GROUP BY name, state, year
    ORDER BY total_by_name DESC
    LIMIT 10
    ),
male_births AS (
    # total births in TX in 2000
    SELECT year, state, COUNT(*) as number_births
    FROM `bigquery-public-data.samples.natality`
    WHERE year = 2000 AND state = 'TX' AND is_male
    GROUP BY year,state
)

SELECT
SUM(total_by_name) top10_names,
male_births.number_births,
ROUND((SUM(total_by_name)/number_births) * 100,2) as percentage
FROM male_names
INNER JOIN male_births ON male_births.state = male_names.state AND male_births.year = male_names.year
GROUP BY male_births.number_births

# important: INNER JOIN conditions are not required here as the queries in the WITH statements
# hardcode the values for TX and the year 2000, but we could remove those filters to generate this
# analysis for any/multiple states, for any/multiple years



# Working with Bigquery GA4 datasets

Execute a query and retrieve all pages that have been viewed by our users on 31st January 2021.

GA4 Table:
`bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20210131`

Collect and show the following dimensions as columns in your results:

* user_pseudo_id
* event_timestamp
* event_name
* page_location (located in event_params)
* page_title (located in event_params)


Ensure to order your results by **user_pseudo_id** and **event_timestamp**


In [ ]:
SELECT
  user_pseudo_id,
  event_timestamp,
  event_name,
  (SELECT params.value.string_value FROM UNNEST(event_params) as params WHERE params.key = 'page_location') AS page_location,
  (SELECT params.value.string_value FROM UNNEST(event_params) as params WHERE params.key = 'page_title') AS page_title
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20210131`
WHERE
  event_name = "page_view"
ORDER BY
  user_pseudo_id ASC,
  event_timestamp ASC



Execute a query to calculate the below key metrics from 1st January 2021 to 31st January 2021.



In [ ]:
SELECT
 COUNT(DISTINCT
   CASE
     WHEN event_name = "page_view" THEN CONCAT(user_pseudo_id, event_timestamp)
   END
 ) AS views,
 COUNT(DISTINCT user_pseudo_id) AS total_users,
 COUNT(DISTINCT
   CASE
     WHEN event_name = "first_visit" THEN user_pseudo_id
   END
 ) AS new_users,
 COUNT(DISTINCT
   CASE
     WHEN event_name = "session_start"
       THEN (SELECT p.value.int_value FROM unnest(event_params) AS p where p.key = "ga_session_id")
   END
 ) AS sessions
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
WHERE
  _TABLE_SUFFIX BETWEEN "20210101" AND "20210131"



# Including user_id count to get total users:
count(distinct
   CASE
     WHEN user_id is not null THEN user_id
     WHEN user_pseudo_id is not null THEN user_pseudo_id
   END
 ) as users_incl_userid,



Execute a query to only extract the top 5 pages for January 2021.



In [ ]:
SELECT
  ep.value.string_value
  ,COUNT(event_name) as views
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
,UNNEST(event_params) as ep
WHERE
  _TABLE_SUFFIX BETWEEN "20210101" AND "20210131"
  AND
  ep.key = "page_title"
  AND
  event_name = "page_view"
  AND
  user_pseudo_id IS NOT NULL
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5


Execute a query to only extract the top 5 products that have been sold between 15 Jan to 31 Jan 2021.

In [ ]:
SELECT
 items.item_name
 ,SUM(items.quantity) as quantity_sold
 ,SUM(items.item_revenue) as product_revenue
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
,unnest(items) as items
WHERE
 _TABLE_SUFFIX BETWEEN "20210115" AND "20210131"
 AND
 event_name = "purchase"
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5
